In [4]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [2]:
from langchain.chat_models import ChatOpenAI

# Different ways to keep API KEY

1. create .env file in the present directory

2. in the cmd type the command in the terminal -> export Openai_api_key value

3. create the enviornment variable in the local machine and export it using -> os.getenv("KEY")

In [8]:
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")

In [7]:
llm = ChatOpenAI(
        openai_api_key=KEY,
        model_name="gpt-3.5-turbo",
        temperature=0.5
    )

In [10]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [13]:
quiz_generation_prompt = PromptTemplate(
                            input_variables=["text", "number", "subject", "tone", "response_json"],
                            template=TEMPLATE
                        )

In [14]:
quiz_generation_chain = LLMChain(
                            llm=llm, prompt=quiz_generation_prompt,
                            output_key="quiz",
                            verbose=True
                        )

In [15]:
TEMPLATE_2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt = PromptTemplate(
                            input_variables=["subject", "quiz"],
                            template=TEMPLATE_2
                        )

In [19]:
quiz_evaluation_chain = LLMChain(
                            llm=llm, prompt=quiz_generation_prompt,
                            output_key="review",
                            verbose=True
                        )

In [21]:
final_integerated_chain = SequentialChain(
                                chains=[quiz_generation_chain,quiz_evaluation_chain],
                                input_variables=["text", "number", "subject", "tone", "response_json","subject"],
                                output_variables=["quiz", "review"],
                                verbose=True
                            )

In [24]:
with open(r"C:\Users\Admin\Documents\Data_Science\GenerativeAI\Generative_AI_Foundations\Projects\mcqGenerator\data.txt", 'r') as file:
    TEXT = file.read()

In [26]:
NUMBER = 5
SUBJECT = "Python GIL"
TONE = "difficult"

In [27]:
with get_openai_callback() as cb:
    response = final_integerated_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The Python Global Interpreter Lock or GIL, in simple words, is a mutex (or a lock) that allows only one thread to hold the control of the Python interpreter.

This means that only one thread can be in a state of execution at any point in time. The impact of the GIL isn’t visible to developers who execute single-threaded programs, but it can be a performance bottleneck in CPU-bound and multi-threaded code.

Since the GIL allows only one thread to execute at a time even in a multi-threaded architecture with more than one CPU core, the GIL has gained a reputation as an “infamous” feature of Python.

In this article you’ll learn how the GIL affects the performance of your Python programs, and how you can mitigate the impact it might have on your code.

What Problem Did the GIL Solve for Python?
Python uses reference counting for memory management. It means that objects created in Pyt

In [28]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:6547
Prompt Tokens:5794
Completion Tokens:753
Total Cost:0.010196999999999998


In [29]:
quiz= response.get("quiz")

In [30]:
quiz = json.loads(quiz)

In [31]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [32]:
quiz_table_data

[{'MCQ': 'What is the purpose of the Global Interpreter Lock (GIL) in Python?',
  'Choices': 'a: To allow multiple threads to execute in parallel | b: To prevent race conditions in memory management | c: To improve the performance of CPU-bound programs | d: To enable garbage collection in Python',
  'Correct': 'b'},
 {'MCQ': 'Which type of programs are most affected by the GIL?',
  'Choices': 'a: I/O-bound programs | b: CPU-bound programs | c: Single-threaded programs | d: Multi-threaded programs',
  'Correct': 'b'},
 {'MCQ': "Why hasn't the GIL been removed from Python?",
  'Choices': 'a: It would cause backward compatibility issues | b: It is a necessary feature for C extensions | c: It is difficult to find an alternative solution | d: It provides performance benefits for single-threaded programs',
  'Correct': 'a'},
 {'MCQ': 'What is one approach to mitigate the impact of the GIL?',
  'Choices': 'a: Using multi-processing instead of multi-threading | b: Using alternative Python inte

In [33]:
quiz = pd.DataFrame(quiz_table_data)

In [34]:
quiz.to_csv("Python_GIL_quiz.csv", index=False)